In [157]:
from time import time
from functools import reduce


def mine_factor(p):
    P = primes(200)
    factors = [[i,1] for i in P if (p-1) % i == 0]
    for i in range(0, len(factors)):
        while (p-1) % (factors[i][0]^(factors[i][1] + 1)) == 0:
            factors[i][1] = factors[i][1] + 1
    F = [i[0] ^ i[1] for i in factors]
    return F

def next_xab(x, a, b, g, h, p):
    
    if Integer(x)%3 == 0:
        x = power_mod(x,2,p)
        a = Integer(2 * a) % (p - 1)
        b = Integer(2 * b) % (p - 1)
    
    elif Integer(x)%3 == 1:
        x = Integer(x * g) % p
        a = Integer(a + 1) % (p - 1)
    
    elif Integer(x)%3 == 2:
        x = Integer(x * h) % p
        b = Integer(b + 1) % (p - 1)
    
    return Integer(x), Integer(a), Integer(b)
    
def Rho(g,h,p):
    x = 1
    a = 0
    b = 0
    i = 1
    
    while True:
        
        X = x
        A,B = a,b
        
        while i < p-1:
            x, a, b = next_xab(x, a, b, g, h, p)
            X, A, B = next_xab(X, A, B, g, h, p)
            X, A, B = next_xab(X, A, B, g, h, p)
            
            if x == X:
                r = (b - B) % (p - 1)
                u = (A - a) 
                
                d = gcd(r, p - 1)   
                
                if d == 1:
                    e = xgcd(r, p-1)[1]
                    x = (u * e) % (p-1)
                    return x
                
                elif d >= 2:
                    s = xgcd(r, p - 1)[1]
                    w = (u * s)%(p - 1)
                    for k in range(0, d):
                        x = ((w / d) + (k * ((p - 1) / d)))
                        if x.is_integer():
                            if pow(g, x ,p) == h:
                                return x    
        i += 1
        a = random.randint(1, p - 1)
        b = random.randint(1, p - 1)
        x = ((g * a) * (h * b)) % p
        X = x
        A, B = a, b

def resids(a, b, p, f):
    x = power_mod(a, Integer((p - 1) / f), p)
    y = power_mod(b, Integer((p - 1) / f), p)
    result = Rho(x, y, p) 
    return result

def getXs(a, b, p, factors):
    residues = []
    for i in factors:
        residues = residues + [resids(a, b, p, i)]
    return residues
    

def chineseRemainder(X, M):
    s = 0
    prod = reduce(lambda a, b: a*b, M)
    for i, j in zip(X,M):
        p = prod // j 
        s += i * xgcd(p, j)[1] * p
    return Integer(s) % prod


p = 0x517C6E0C07F34CCFF1BF27D2C1C93E608AD2B49D6DEB0A5E419B25DC1B47208FEABF6771C6A8B018EFCCEFDE9505CB5CD1F8C0539362C4716EABDA7FBCF3294BE1D
g = 0x2C9587F5FE406EE27111F7BA768F914DFC4041D98F0108A8F9E9EF517E219B89516F6775EF0A9271631A85CBBA7D70822659728ABD369CCAF61A2DA774346C804C0
h = 0x1A6D6F87F388CD9B6C98061BA20AE93EF023F28FA73FF12C7B1BAAF62D091A48ABD152A40F68932AB5EBB6CF6E6AFE02DB36F3B7C02A6DE15B66BC8038D72415EED

F = [37**2, 181**2]   ## modulniki do CRT
start = time()
X = getXs(g, h, p, F) ## reszty kwadratowe do CRT
result = chineseRemainder(X, F)
print("Wynik dzialania programu wynosi: ")
print(hex(result))
print("Sprawdzenie")
print(hex(power_mod(g, result, p)))
end = time()
print("Czas wykonania: "+str(end-start))
if power_mod(g, result, p) == h:
    print("Udalo sie")
else:
    print("Nie udalo sie")

Wynik dzialania programu wynosi: 
0xa225b5
Sprawdzenie
0x2e49e42867ed6fcaa1d15fcc2d5eb27adb91824b69c97307b67185cb849c9e7c7f5b772a2157ad3ba19e2755108910ef2f3d98b1cf81a9b3a76113f6a0f75b7c5af
Czas wykonania: 0.013225793838500977
Nie udalo sie


In [153]:
g, h, p = 17, 49, 107
x = Rho(g, h, p)
print(x)
print()
if pow(g, x, p) == h:
    print(pow(g, x, p))
    print("Funkcja Rho() wykonala poprawne obliczenia")

98

49
Funkcja Rho() wykonala poprawne obliczenia


In [95]:
def shanks(g, h, p):
    m =  ceil(sqrt(p-1))
    
    L = dict([(i, pow(g, i, p)) for i in range(0, m)])
    l = {k: v for k, v in sorted(L.items(), key=lambda item: item[1])} #posortowany slownik
    
    gmInv = pow(g, -m, p)
    j = 0
    
    while True:
        target = h * power_mod(gmInv, j, p)
        for i in l:
            if l[i] == target:
                return ((j*m) + i) % p
            else:    
                j +=1